<a href="https://colab.research.google.com/github/sofial47/senalesysistemas/blob/main/Cuadernoyoutube_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#cargar datos desde drive acceso libre
#https://docs.google.com/spreadsheets/d/1o9Bw_W0Au5LgqMGteDRROf0avbmyMCyy/edit?usp=sharing&ouid=108533457420906937206&rtpof=true&sd=true
#1o9Bw_W0Au5LgqMGteDRROf0avbmyMCyy
FILEID = "1o9Bw_W0Au5LgqMGteDRROf0avbmyMCyy" #"1DxI5wQpqEWksw2BqJnG7n0IgNQ_xByX-"

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O  canciones.xlsx && rm -rf /tmp/cookies.txt
#!unzip -o codigos.zip
!dir

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_ = 'canciones.xlsx'#leer archivo xlsx con link, band, type
X  = pd.read_excel(file_)
X#imprimir filas iniciales

In [ ]:
#instalar librerias necesarias para descargar audios youtube
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile

# Paquete de carga de cookies para el control anti-bots captcha
!pip install browser-cookie3

import os
import yt_dlp as youtube_dl
import browser_cookie3

try:
    cookies = browser_cookie3.firefox()
except:
    print("No se pueden descargar cookies desde firefox. Intentando Chrome...")
    try:
        cookies = browser_cookie3.chrome()
    except:
        print("No se pueden descargar cookies desde Chrome. Por favor asegúrate de estar logueado en Youtube desde tu navegador.")
        cookies = None

#funcion para descargar mp3 desde youtube
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':f'{name}.mp3',
    }
    if cookies:
        options['cookiefile'] = None
        options['cookiejar'] = cookies;

    with youtube_dl.YoutubeDL(options) as ydl:
        try:
            video_info = ydl.extract_info(video_url, download=False)
            ydl.download([video_info['webpage_url']])
            print("Download complete... {}".format(filename))
        except Exception as e:
            print(f"Error descargando {video_url}: {e}")

In [ ]:
import subprocess
import os

#crear carpeta con resultados
try:
  os.mkdir('results')
except:
  print("Carpeta results ya existe")

#recorrer excel con videos
N, P = X.shape
Ns = N * 5 #cantidad de segmentos por cancion

for n in range(N):
    print(f"video {n+1} de {N}")
    print(f"link: {X.loc[n,'link']}\n")
    print(f"band: {X.loc[n,'band']}\n")
    print(f"type: {X.loc[n,'type']}\n")
    #ruta video n-th
    name_ = 'results/'+X.loc[n,'band']+"_"+str(n)+"_"+str(X.loc[n,'type_num']) # #video+nombre+tipo de genero musical
    #descargar mp3 desde youtube
    try: # Added try block to catch download errors
        download_ytvid_as_mp3(X.loc[n,'link'],name_)
        #convertir a .wav
        subprocess.call(['ffmpeg','-y', '-i', name_+'.mp3',
                       name_+'.wav'])
    except youtube_dl.utils.DownloadError as e: # Catch the specific DownloadError
        print(f"Skipping video {X.loc[n,'link']} due to download error: {e}") # Print a message and skip

In [ ]:
#cargar .wavs y partir audios
#lista archivos .wav
path = 'results/'
wav_files = [f for f in os.listdir(path) if f.endswith('.wav')]
wav_files

In [ ]:
#import soundfile as sf # para instalar pip install soundfile
#from scipy.signal import resample_poly


#leer archivos y crear np.array audios
#fs = 48000
#tl = np.array([30,40,50,60,70,80]) #puntos lectura
#ts = 5 #t segmento
#Ns = len(wav_files)*len(tl) #cantidad segmentos
#x_t = np.zeros((Ns,int(ts*fs),2)) #Ns segmentos, cantidad de muestras, 2 canales (stereo)
#label = np.zeros((Ns,1)) #vector tipo de genero
#type_c = X['type'].unique()
#name_c = []
#leer archivos wav
#i = 0
#for name in wav_files:#lectura audio .wav
 #   x, fs_i = sf.read(path+name)
  #  for ti in tl: #segmentos de tiempo
   #     print(name,'x',x.shape,'fs actual:',fs_i)
    #    x_t[i]=x[int(fs*ti):int(fs*(ti+ts)),:]
        #xc = x[int(fs*ti):int(fs*(ti+ts)),:]
        #if fs_i < fs:
          # Resample using resample_poly
         # gcd = np.gcd(fs, fs_i)
          #up = fs // gcd
         # down = fs_i // gcd
          #xc = resample_poly(x, up=up, down=down)
        #x_t[i] = xc
     #   label[i] = int(name[-5]) #tipo de genero
      #  name_c += [name[:-6]]
       # print(f"{i} lectura: {name}; segundo {ti}:{ti+ts}; tipo música: {type_c[int(label[i])-1]}")
        #i+=1
#x_t.shape

In [ ]:
import soundfile as sf # para instalar pip install soundfile
from scipy.signal import resample_poly
import numpy as np # Ensure numpy is imported for np.gcd and other operations

#leer archivos y crear np.array audios
fs = 48000
tl = np.array([30,40,50,60,70,80]) #puntos lectura, cambie este
#tl=np.arange(0,250,5)
ts = 5 #t segmento
Ns = len(wav_files)*len(tl) #cantidad segmentos
x_t = np.zeros((Ns,int(ts*fs),2)) #Ns segmentos, cantidad de muestras, 2 canales (stereo)
label = np.zeros((Ns,1)) #vector tipo de genero
type_c = X['type'].unique()
name_c = []
#leer archivos wav
i = 0
for name in wav_files:#lectura audio .wav
    x, fs_i = sf.read(path+name)

    # Verificar la forma de x
    if x.ndim != 2 or x.shape[1] != 2:
        print(f"Advertencia: El archivo {name} no tiene 2 canales. Shape: {x.shape}. Saltando.")
        continue # Saltar a la siguiente iteración si no es estéreo de 2 canales

    for ti in tl: #segmentos de tiempo
        print(f"Procesando {name}, segundo {ti}:{ti+ts}, tipo música: {type_c[int(label[i])-1]}")
        print(f"  Shape de audio completo: {x.shape}, fs actual: {fs_i}")

        # Calcular los índices de inicio y fin del segmento en la frecuencia de muestreo original
        start_sample = int(fs_i * ti)
        end_sample = int(fs_i * (ti + ts))

        # Extraer el segmento en la frecuencia de muestreo original
        xc = x[start_sample:end_sample,:]
        print(f"  Shape del segmento extraído (antes de remuestreo): {xc.shape}")

        # Check if resampling is needed
        # Comprobar si se necesita remuestreo
        if fs_i != fs:
            print(f"  Remuestreando de {fs_i} Hz a {fs} Hz.")
            # Resample using resample_poly
            # Remuestrear usando resample_poly
            # Calculate the ratio for resampling
            # Calcular la relación para el remuestreo
            gcd = np.gcd(fs, fs_i)
            up = fs // gcd
            down = fs_i // gcd
            # Apply resampling
            # Aplicar remuestreo
            xc_resampled = resample_poly(xc, up=up, down=down)

            # Verificar la forma del segmento remuestreado
            print(f"  Shape del segmento remuestreado: {xc_resampled.shape}")

            # Asegurarse de que el segmento remuestreado tenga el número correcto de muestras
            # Truncamos o rellenamos con ceros si es necesario para asegurar la forma exacta
            target_samples = int(ts * fs)
            if xc_resampled.shape[0] != target_samples:
                print(f"  Corrigiendo el número de muestras remuestreadas de {xc_resampled.shape[0]} a {target_samples}.")
                temp_xc = np.zeros((target_samples, 2))
                copy_samples = min(xc_resampled.shape[0], target_samples)
                temp_xc[:copy_samples, :] = xc_resampled[:copy_samples, :]
                x_t[i] = temp_xc
            else:
                 x_t[i] = xc_resampled
        else:
            # If sampling rates are the same, just assign the segment
            # Si las frecuencias de muestreo son las mismas, simplemente asignar el segmento
            # Asegurarse de que el segmento extraído tenga la forma correcta
            target_samples = int(ts * fs)
            if xc.shape[0] != target_samples:
                 print(f"  Advertencia: El segmento extraído a la misma fs ({fs_i}) no tiene {target_samples} muestras. Shape: {xc.shape}. Rellenando/Truncando.")
                 temp_xc = np.zeros((target_samples, 2))
                 copy_samples = min(xc.shape[0], target_samples)
                 temp_xc[:copy_samples, :] = xc[:copy_samples, :]
                 x_t[i] = temp_xc
            else:
                x_t[i] = xc


        # Solo asignamos la etiqueta si el segmento fue procesado (no saltado por no tener 2 canales)
        # name[-5] asume que el tipo está en el 5º carácter desde el final
        # Asegurarse de que este índice es correcto y que el nombre tiene suficientes caracteres
        try:
            label[i] = int(name[-5]) #tipo de genero
        except (ValueError, IndexError):
             print(f"Advertencia: No se pudo extraer el tipo de género del nombre del archivo {name}. Asegúrese de que el formato del nombre sea correcto.")
             # Decide cómo manejar esto. Podrías asignar un valor por defecto o saltar este segmento/archivo.
             # Por ahora, asignaremos un 0 y mostraremos una advertencia.
             label[i] = 0


        name_c += [name[:-6]]
        # Ajustamos el print para usar la etiqueta asignada, en caso de error
        label_val = int(label[i])
        type_display = "Desconocido"
        if 0 < label_val <= len(type_c):
             type_display = type_c[label_val-1]

        print(f"{i} lectura: {name_c[-1]}; segundo {ti}:{ti+ts}; tipo música: {type_display}")
        i+=1

# Redimensionar label para que coincida con el número actual de segmentos procesados, en caso de que algunos se saltaran
if i < Ns:
    print(f"Solo se procesaron {i} segmentos de los {Ns} esperados.")
    x_t = x_t[:i]
    label = label[:i]

x_t.shape
print(f"Total de segmentos procesados: {x_t.shape[0]}")

In [ ]:
from IPython.display import Audio #reproducir segmento
i = 0
Audio(x_t[i].T,rate=fs)

In [ ]:
#calculo de fourier
vf = np.fft.rfftfreq(x_t.shape[1],1/fs) #calculo vector de frecuencias
Xw = np.fft.rfft(x_t,axis=1).mean(axis=-1) #transformada rapida de Fourier para señal Real a lo largo del tiempo (axis=1) y se promedian los dos canales
Xw.shape

In [ ]:
#grafica tiempo y fourier
plt.plot(np.arange(0,ts,1/fs),x_t.mean(axis=-1).T) #se promedian los dos canales stereo
plt.xlabel('t[s]')
plt.ylabel('x(t)')
plt.show()

In [ ]:
plt.plot(vf,abs(Xw).T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

In [ ]:
#se normalizan espectros entre 0 y 1 para evitar inconsistencias por ampliltudes máximas
from sklearn.preprocessing import MinMaxScaler
sca = MinMaxScaler()
Xw_ = sca.fit_transform(abs(Xw).T).T

plt.plot(vf,Xw_.T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

#en dB
plt.plot(vf,(20*np.log10(Xw_+1e-10)).T) # se suma 1e-10 para evitar discontinuidad del log
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)| dB')
plt.show()

**Nota**: Generalmente el espectro se presenta en [decibeles [dB]](https://es.wikipedia.org/wiki/Decibelio)

## Visualización de los datos en 2D

In [ ]:
from umap import UMAP
#visualización de datos
n_neighbors = int(2*np.sqrt(Xw_.shape[0]))
sca_ = MinMaxScaler()

red_ = UMAP(n_components=2,n_neighbors=n_neighbors)
X_2D = sca_.fit_transform(red_.fit_transform(Xw_))

In [ ]:
#graficar separabilidad 2D
plt.scatter(X_2D[:,0],X_2D[:,1],c=label)
plt.colorbar()
plt.show()

color_ = ["b","y"]
#nombre cancion
plt.scatter(X_2D[:,0],X_2D[:,1],c=label,s=1)
for i, tex in enumerate(name_c):
    #print(f"{i} {tex}")
    plt.text(X_2D[i,0]*1.025,X_2D[i,1]*1.025, tex[:-2]+"_"+str(i), fontsize=6,color=color_[int(label[i]-1)])

plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.show()

In [ ]:
#reproducir audio
i =43
Audio(x_t[i].T,rate=fs)

# Predicción del género musical sobre espectro de Fourier

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Crear una instancia del clasificador RandomForest
model = KNeighborsClassifier(n_neighbors=5)

# Realizar la validación cruzada
# Xw_ son tus características y label son tus etiquetas
# cv define el número folds para la validación cruzada
scores = cross_val_score(model, Xw_, label.ravel(), cv=5,scoring='accuracy')

# Imprimir los resultados de la validación cruzada
print("Acierto para cada Fold:", scores)
print("Acierto promedio:", scores.mean())
print("Desviación estándar del Acierto:", scores.std())

#entrenar modelo para predecir nuevas canciones
model.fit(Xw_, label.ravel())

In [ ]:
import joblib
#guardar modelo
try:
  os.mkdir('modelo')
except:
  print("Carpeta modelo ya existe")

filename_ = 'modelo/phonk_vs_clasica'
model_ ={'Xw_':Xw_, 'label' : label, 'name_c' : name_c, 'vf':vf,'fs':fs, 'modelo':model,'type':X['type'].unique()}
joblib.dump(model_,filename_+".pkl")


In [ ]:
#descargar modelo
#guardar resultados
from google.colab import files
from datetime import date, datetime
import shutil
#guardar resultados
namefile = str(datetime.now().strftime("%Y_%m_%d_%H_%M_%d"))+'modelo'
shutil.make_archive(namefile, 'zip', 'modelo')
files.download(namefile+'.zip')

#el archivo .zip puede cargarse en drive y utilizarse en otro cuaderno para detectar género musical de nuevos segmentos

# Para una nueva canción

In [ ]:
#cargar modelo
my_model_loaded = joblib.load(filename_+".pkl")
my_model_loaded.keys()

In [ ]:
#evaluar para un segmento en fft normalizado

pred_ = my_model_loaded['modelo'].predict(Xw_[0].reshape(1,-1))[0]
print('Etiqueta estimada: ',my_model_loaded['type'][int(pred_-1)])
print('Etiqueta orignal: ',my_model_loaded['type'][int(my_model_loaded['label'][0][0])-1])

In [ ]:
# %% [markdown]
# #Detección de género para un nuevo registro musical
## %%
import soundfile as sf
from scipy.signal import resample_poly
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# --- Paso 1: Simular o cargar un nuevo audio de 5 segundos ---
# Opción 1: Simular un audio de 5 segundos (para prueba)
# Aquí creamos un audio de prueba con ruido aleatorio.
# En un caso real, cargarías un archivo .wav o .mp3
fs = my_model_loaded['fs'] # Usar la frecuencia de muestreo del modelo
ts = 5 # Duración del segmento en segundos
new_audio_samples = int(ts * fs)
# Simular audio estéreo de 5 segundos con ruido aleatorio
new_audio_data = np.random.rand(new_audio_samples, 2) * 0.1

# Opción 2: Cargar un archivo de audio real (descomentar y modificar la ruta)
# try:
#     # Reemplaza 'ruta/a/tu/nuevo_audio.wav' con la ruta real de tu archivo de audio
#     new_audio_data, fs_new = sf.read('ruta/a/tu/nuevo_audio.wav')
#
#     # Asegurarse de que el audio tiene 2 canales (estéreo)
#     if new_audio_data.ndim != 2 or new_audio_data.shape[1] != 2:
#         print("Advertencia: El nuevo archivo de audio no tiene 2 canales. Intentando convertir a estéreo (puede no funcionar para todos los casos).")
#         # Intenta expandir a 2 canales si es mono
#         if new_audio_data.ndim == 1:
#             new_audio_data = np.column_stack((new_audio_data, new_audio_data))
#         else:
#              raise ValueError("El archivo de audio tiene un formato de canal inesperado.")
#
#     # Asegurarse de que la duración sea exactamente 5 segundos o ajustarla
#     target_samples = int(ts * fs)
#     if new_audio_data.shape[0] != target_samples:
#         print(f"Advertencia: El nuevo audio no tiene exactamente {ts} segundos ({target_samples} muestras a {fs} Hz). Ajustando...")
#         temp_audio = np.zeros((target_samples, 2))
#         copy_samples = min(new_audio_data.shape[0], target_samples)
#         temp_audio[:copy_samples, :] = new_audio_data[:copy_samples, :]
#         new_audio_data = temp_audio
#
#     # Remuestrear si la frecuencia de muestreo es diferente
#     if fs_new != fs:
#         print(f"Remuestreando el nuevo audio de {fs_new} Hz a {fs} Hz.")
#         gcd = np.gcd(fs, fs_new)
#         up = fs // gcd
#         down = fs_new // gcd
#         new_audio_data = resample_poly(new_audio_data, up=up, down=down)
#         # Después del remuestreo, asegurarse de que el número de muestras sea correcto
#         if new_audio_data.shape[0] != target_samples:
#              print(f"Corrección post-remuestreo: ajustando número de muestras de {new_audio_data.shape[0]} a {target_samples}.")
#              temp_audio = np.zeros((target_samples, 2))
#              copy_samples = min(new_audio_data.shape[0], target_samples)
#              temp_audio[:copy_samples, :] = new_audio_data[:copy_samples, :]
#              new_audio_data = temp_audio
#
# except FileNotFoundError:
#     print("Error: El archivo de audio no se encontró. Usando datos simulados para la demostración.")
#     # Si el archivo no se encuentra, vuelve a la opción simulada
#     fs = my_model_loaded['fs']
#     ts = 5
#     new_audio_samples = int(ts * fs)
#     new_audio_data = np.random.rand(new_audio_samples, 2) * 0.1
# except Exception as e:
#     print(f"Error al cargar o procesar el nuevo archivo de audio: {e}. Usando datos simulados para la demostración.")
#     # Si hay otro error, vuelve a la opción simulada
#     fs = my_model_loaded['fs']
#     ts = 5
#     new_audio_samples = int(ts * fs)
#     new_audio_data = np.random.rand(new_audio_samples, 2) * 0.1

# --- Paso 2: Preprocesar el nuevo audio ---

# Calcular la transformada de Fourier de magnitud
# El .mean(axis=-1) promedia los canales estéreo
# Note: rfft returns frequencies up to fs/2. The number of bins is N/2 + 1
# For a 5-second segment at 48000 Hz, N = 5 * 48000 = 240000
# Number of frequency bins = 240000 / 2 + 1 = 120001
new_Xw = np.fft.rfft(new_audio_data, axis=0).mean(axis=-1)
new_Xw_magnitude = abs(new_Xw)

# Normalizar el espectro de Fourier usando el escalador ajustado del entrenamiento
# Necesitamos que el escalador esté disponible. Si lo cargaste con el modelo, úsalo.
# Si no, deberías haberlo guardado y cargado también, o reajustarlo (aunque reajustar no es lo ideal
# ya que la normalización debe ser consistente con los datos de entrenamiento).
# Asumiremos que el escalador se puede re-crear y ajustar sobre los datos de entrenamiento
# si no fue guardado explícitamente en el modelo. Sin embargo, la forma correcta sería
# guardar el objeto `sca` (MinMaxScaler) durante el entrenamiento y cargarlo aquí.

# Para este ejemplo, re-ajustaremos un nuevo escalador en los datos de entrenamiento
# cargados del modelo. En un escenario de producción, guardar y cargar el escalador es mejor.
sca_loaded = MinMaxScaler()
# CORRECT FIT: Fit the scaler on the training data (Xw_) where rows are samples
# and columns are features (frequency bins).
# FITTING shape is (Ns, 120001)
sca_loaded.fit(my_model_loaded['Xw_']) # Ajustar el escalador en los datos de entrenamiento

# Normalizar el nuevo espectro
# El escalador espera una entrada 2D (n_muestras, n_características).
# Aquí tenemos solo 1 muestra (el nuevo audio), así que lo reshapeamos.
# The input to transform should have the shape (n_samples, n_features)
# The new_Xw_magnitude has shape (120001,). Reshaping to (1, 120001)
new_Xw_normalized = sca_loaded.transform(new_Xw_magnitude.reshape(1, -1))

# --- Paso 3: Realizar la predicción con el modelo KNN cargado ---

# Usar el modelo cargado para predecir
predicted_label_num = my_model_loaded['modelo'].predict(new_Xw_normalized)[0]

# Obtener el nombre del género a partir del número predicho
# Recordar que las etiquetas en label son 1, 2, etc., y type_c tiene los nombres
predicted_genre = my_model_loaded['type'][int(predicted_label_num) - 1]

# Imprimir el resultado
print(f"\nEl género musical predicho para el nuevo registro es: {predicted_genre}")

# Opcional: Reproducir el nuevo audio (si usaste la opción de cargar un archivo)
# from IPython.display import Audio
# Audio(new_audio_data.T, rate=fs)